In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression

In [2]:
spark = SparkSession.builder.appName('logreg').getOrCreate()

In [3]:
# Get Data
df = spark.read.format('libsvm').load('sample_libsvm_data.txt')

In [4]:
df.show() # Label is 0 or 1 -> BINARY CLASSIFICATION

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [5]:
# Initiate the Logistic Regression Model
logReg = LogisticRegression()

In [6]:
# Fit the model
logRegModel = logReg.fit(df)

In [8]:
# View results
results = logRegModel.summary

In [10]:
results.predictions.printSchema()
# label         -> real value
# features      -> features to build the model
# rawPrediction -> prediction of model before fitting to data
# probability   -> prob. of rawPrediction matching label
# prediction    -> prediction of model after fitting to data

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [11]:
results.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[19.8534775947478...|[0.99999999761359...|       0.0|
|  1.0|(692,[158,159,160...|[-20.377398194908...|[1.41321555111048...|       1.0|
|  1.0|(692,[124,125,126...|[-27.401459284891...|[1.25804865126969...|       1.0|
|  1.0|(692,[152,153,154...|[-18.862741612668...|[6.42710509170264...|       1.0|
|  1.0|(692,[151,152,153...|[-20.483011833009...|[1.27157209200596...|       1.0|
|  0.0|(692,[129,130,131...|[19.8506078990277...|[0.99999999760673...|       0.0|
|  1.0|(692,[158,159,160...|[-20.337256674833...|[1.47109814695572...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.595579753418...|[3.08850168102604...|       1.0|
|  0.0|(692,[154,155,156...|[19.2708803215612...|[0.99999999572670...|       0.0|
|  0.0|(692,[127

In [12]:
# EVALUATORS
# Let's run an evaluator on test data

In [13]:
train_data,test_data = df.randomSplit([0.7,0.3])

In [14]:
final_model = LogisticRegression()

In [16]:
fit_final = final_model.fit(train_data)

In [17]:
prediction_and_labels = fit_final.evaluate(test_data)

In [19]:
prediction_and_labels.predictions.show()
# Same but not data is split into train and test

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[95,96,97,12...|[21.9246393887383...|[0.99999999969921...|       0.0|
|  0.0|(692,[122,123,148...|[19.5771193359086...|[0.99999999685395...|       0.0|
|  0.0|(692,[124,125,126...|[31.8574407548028...|[0.99999999999998...|       0.0|
|  0.0|(692,[124,125,126...|[16.7209298007311...|[0.99999994527416...|       0.0|
|  0.0|(692,[124,125,126...|[26.0467349849349...|[0.99999999999512...|       0.0|
|  0.0|(692,[126,127,128...|[24.2820756719009...|[0.99999999997152...|       0.0|
|  0.0|(692,[126,127,128...|[33.1543917507737...|[0.99999999999999...|       0.0|
|  0.0|(692,[127,128,129...|[20.5733753223147...|[0.99999999883829...|       0.0|
|  0.0|(692,[127,128,129...|[23.6350981635329...|[0.99999999994562...|       0.0|
|  0.0|(692,[152

In [20]:
from pyspark.ml.evaluation import (BinaryClassificationEvaluator,
                                   MulticlassClassificationEvaluator)

In [21]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction')

In [22]:
myFinalROC = evaluator.evaluate(prediction_and_labels.predictions)

In [23]:
myFinalROC # Perfect Fit. Predicted everything

1.0